In [69]:
import pandas as pd
import numpy as np
from dune_client.client import DuneClient
from flipside import Flipside
import plotly
import datetime as dt
from dotenv import load_dotenv
from prophet import Prophet
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


# import yfinance as yf

from utils import flipside_api_results
from sql import trader_classifier_query

In [2]:
load_dotenv()

True

In [3]:
dune_api_key = os.getenv('DUNE_API_KEY')
flipside_api_key = os.getenv('FLIPSIDE_API_KEY')
dune = DuneClient(dune_api_key)
print(dune_api_key, flipside_api_key)

CPh7RV9x58ODbTuhYEA9lnpD4NkIvyEi 4604d4d2-ccf3-4864-90b4-db6bf13c663b


# Data Retrieval 

## Dune Queries

Uniswap Arbitrum Pool Data
https://dune.com/queries/3930878

Arbitrum Gas Data
https://dune.com/queries/3931017

In [42]:
def dune_api_results(query_num, save_csv=False, csv_path=None):
    results = dune.get_latest_result(query_num)
    df = pd.DataFrame(results.result.rows)

    if save_csv and csv_path:
        df.to_csv(csv_path, index=False)
    return df


arb_pool_path = 'data/arb_pool_data.csv'
arb_pool_df = dune_api_results(3930878, True, arb_pool_path)
arb_pool_df

In [45]:
arb_pool_path = 'data/arb_pool_data.csv'
arb_pool_df = pd.read_csv(arb_pool_path)
arb_pool_df.columns = [f'arbitrum_{col}' if col != 'day' else col for col in arb_pool_df.columns]
print(arb_pool_df.columns)

Index(['arbitrum_avg_liquidity', 'day', 'arbitrum_fee_apr',
       'arbitrum_fee_tier', 'arbitrum_fees_usd', 'arbitrum_lp_addr',
       'arbitrum_num_trades', 'arbitrum_token_pair', 'arbitrum_tvl_usd',
       'arbitrum_volume_to_tvl', 'arbitrum_volume_usd'],
      dtype='object')


arbitrum_gas = dune.get_latest_result(3931017)
arbitrum_gas_df = pd.DataFrame(arbitrum_gas.result.rows)
arbitrum_gas_path = 'data/arb_gas.csv'
arbitrum_gas_df.to_csv(arbitrum_gas_path, index=False)

In [25]:
arbitrum_gas_path = 'data/arb_gas.csv'
arbitrum_gas_df = pd.read_csv(arbitrum_gas_path)
if 'Unnamed: 0' in arbitrum_gas_df.columns:
    arbitrum_gas_df = arbitrum_gas_df.drop('Unnamed: 0', axis=1)
arbitrum_gas_df.columns = [f'arbitrum_{col}' if col != 'dt' else col for col in arbitrum_gas_df.columns]
arbitrum_gas_df.columns


,dt,gas_usd_per_tx,median_gas_usd
0,2024-04-22 00:00:00.000 UTC,0.010387,0.006123
1,2024-04-22 01:00:00.000 UTC,0.010970,0.006445
2,2024-04-22 02:00:00.000 UTC,0.009510,0.005550
3,2024-04-22 03:00:00.000 UTC,0.010174,0.005812
4,2024-04-22 04:00:00.000 UTC,0.009947,0.006139
...,...,...,...
2157,2024-07-20 21:00:00.000 UTC,0.009192,0.005052
2158,2024-07-20 22:00:00.000 UTC,0.010094,0.005563
2159,2024-07-20 23:00:00.000 UTC,0.009521,0.005383
2160,2024-07-21 00:00:00.000 UTC,0.009502,0.005291


Uniswap Optimism Pool Data
https://dune.com/queries/3930989

Optimism Gas Data 
https://dune.com/queries/3930989

op_pool_path = 'data/op_pool_data.csv'
op_pool_df = dune_api_results(3930989, True, op_pool_path)
op_pool_df

In [47]:
op_pool_path = 'data/op_pool_data.csv'
op_pool_df = pd.read_csv(op_pool_path)
op_pool_df.columns = [f'optimism_{col}' if col != 'day' else col for col in op_pool_df.columns]
print(op_pool_df.columns)

Index(['optimism_avg_liquidity', 'day', 'optimism_fee_apr',
       'optimism_fee_tier', 'optimism_fees_usd', 'optimism_lp_addr',
       'optimism_num_trades', 'optimism_token_pair', 'optimism_tvl_usd',
       'optimism_volume_to_tvl', 'optimism_volume_usd'],
      dtype='object')


optimism_gas = dune.get_latest_result_dataframe(3931019)
optimism_gas_path = 'data/op_gas.csv'
optimism_gas.to_csv(optimism_gas_path, index=False)

In [36]:
optimism_gas_path = 'data/op_gas.csv'
optimism_gas_df = pd.read_csv(optimism_gas_path)
optimism_gas_df = optimism_gas_df.drop('_col3', axis=1) if '_col3' in optimism_gas_df.columns else optimism_gas_df
optimism_gas_df.columns = [f'optimism_{col}' if col != 'dt' else col for col in optimism_gas_df.columns]
optimism_gas_df.columns

Index(['dt', 'optimism_gas_usd_per_tx', 'optimism_median_gas_usd'], dtype='object')

Uniswap Base Pool Data
https://dune.com/queries/3930954

Base Gas Data
https://dune.com/queries/3931021

base_pool_path = 'data/base_pool_data.csv'
base_pool_df = dune_api_results(3930954, True, base_pool_path)
base_pool_df

In [49]:
base_pool_path = 'data/base_pool_data.csv'
base_pool_df = pd.read_csv(base_pool_path)
base_pool_df.columns = [f'base_{col}' if col != 'day' else col for col in base_pool_df.columns]
print(base_pool_df.columns)

Index(['base_avg_liquidity', 'day', 'base_fee_apr', 'base_fee_tier',
       'base_fees_usd', 'base_lp_addr', 'base_num_trades', 'base_token_pair',
       'base_tvl_usd', 'base_volume_to_tvl', 'base_volume_usd'],
      dtype='object')


base_gas = dune.get_latest_result_dataframe(3931021)
base_gas_path = 'data/base_gas.csv'
base_gas.to_csv(base_gas_path, index=False)

In [39]:
base_gas_path = 'data/base_gas.csv'
base_gas_df = pd.read_csv(base_gas_path)
base_gas_df.columns = [f'base_{col}' if col != 'dt' else col for col in base_gas_df.columns]
base_gas_df.columns

Index(['dt', 'base_gas_usd_per_tx', 'base_median_gas_usd'], dtype='object')

## Flipside Queries 

Trader Classifier - Arbitrum Data
https://flipsidecrypto.xyz/Brandyn/q/7NlPxrKU5KQb/2024-07-20-06-36-pm

In [4]:
classifier_data_path = 'data/classifier.csv'
trader_classifier_data = flipside_api_results(trader_classifier_query, flipside_api_key)
print(trader_classifier_data)
trader_classifier_data.to_csv(classifier_data_path, index=False)

        trader_type                        dt  tx_count  total_volume_usd  \
0      Professional  2024-07-21T15:00:00.000Z       928        1047810.93   
1            Retail  2024-07-21T15:00:00.000Z       176          91856.49   
2      Professional  2024-07-21T14:00:00.000Z      7552       12913378.38   
3            Retail  2024-07-21T14:00:00.000Z      1790         294765.21   
4      Professional  2024-07-21T13:00:00.000Z      4472        5883981.27   
...             ...                       ...       ...               ...   
18553        Retail  2023-07-01T02:00:00.000Z      1170         278669.36   
18554  Professional  2023-07-01T01:00:00.000Z      4263       12052488.42   
18555        Retail  2023-07-01T01:00:00.000Z      1254         301856.49   
18556  Professional  2023-07-01T00:00:00.000Z      4124       11335892.35   
18557        Retail  2023-07-01T00:00:00.000Z      1084         402822.79   

       avg_order_size_usd  unique_contracts  __row_index  
0             11

In [6]:
trader_classifier_df = pd.read_csv(classifier_data_path)
trader_classifier_df.drop(columns=['__row_index'], inplace=True)
trader_classifier_df.head()

,trader_type,dt,tx_count,total_volume_usd,avg_order_size_usd,unique_contracts
0,Professional,2024-07-21T15:00:00.000Z,928,1047810.93,1183.967153,104
1,Retail,2024-07-21T15:00:00.000Z,176,91856.49,540.332294,60
2,Professional,2024-07-21T14:00:00.000Z,7552,12913378.38,1766.536030,252
3,Retail,2024-07-21T14:00:00.000Z,1790,294765.21,170.680492,196
4,Professional,2024-07-21T13:00:00.000Z,4472,5883981.27,1367.731583,211


# Data Cleaning/Processing

Each token pair has several fee tiers and some have differente lp addresses.  

Can aggregate metrics to token pair and/or seperate each fee-tier out, aggregate by addresses

# Data Analysis

## Arbitrum

### Summary Statistics

In [51]:
print(arb_pool_df.describe())

       arbitrum_avg_liquidity  arbitrum_fee_apr  arbitrum_fees_usd  \
count            3.468000e+03      3.468000e+03       3.468000e+03   
mean             1.782774e+24      1.458773e-02       2.966866e+02   
std              7.617222e+24      5.095527e-02       7.349982e+02   
min              9.140293e+07      3.241851e-13       6.002412e-15   
25%              6.812052e+15      8.222112e-04       2.920990e-01   
50%              1.903847e+18      4.227947e-03       1.321334e+01   
75%              2.049945e+22      1.363400e-02       2.495379e+02   
max              4.600563e+25      1.606219e+00       8.186259e+03   

       arbitrum_num_trades  arbitrum_tvl_usd  arbitrum_volume_to_tvl  \
count          3468.000000      3.468000e+03            3.468000e+03   
mean            190.908304      9.310402e+06            5.508794e-02   
std             338.143117      1.496615e+07            1.409350e-01   
min               1.000000      6.708087e+00            2.982674e-13   
25%      

In [52]:
print(arbitrum_gas_df.describe())

       arbitrum_gas_usd_per_tx  arbitrum_median_gas_usd
count              2162.000000              2162.000000
mean                  0.023634                 0.012860
std                   0.221630                 0.129315
min                   0.004397                 0.001491
25%                   0.006888                 0.004532
50%                   0.008472                 0.005564
75%                   0.010704                 0.007182
max                   6.944713                 4.181090


### Data Visualizations

In [92]:
arb_pool_df

,arbitrum_avg_liquidity,day,arbitrum_fee_apr,arbitrum_fee_tier,arbitrum_fees_usd,arbitrum_lp_addr,arbitrum_num_trades,arbitrum_token_pair,arbitrum_tvl_usd,arbitrum_volume_to_tvl,arbitrum_volume_usd
0,1.854139e+23,2024-07-14 00:00:00.000 UTC,3.781496e-02,0.30%,386.685410,0x4cef551255ec96d89fec975446301b5c4e164c59,119,WETH-ZRO,3.732390e+06,3.453421e-02,1.288951e+05
1,2.075720e+16,2024-07-14 00:00:00.000 UTC,1.591761e-02,0.01%,3.056607,0x6f38e884725a116c9c7fbf208e79fe8828a2595f,287,USDC-WETH,7.008980e+04,4.360988e-01,3.056607e+04
2,1.965666e+12,2024-07-14 00:00:00.000 UTC,1.469532e-02,0.05%,412.274720,0x0e4831319a50228b9e450861297ab92dee15b44f,466,USDC-WBTC,1.024001e+07,8.052231e-02,8.245494e+05
3,4.911041e+18,2024-07-14 00:00:00.000 UTC,1.290974e-02,0.05%,1705.827484,0xc6962004f452be9203591991d15f6b388e09e8d0,845,USDC-WETH,4.822924e+07,7.073831e-02,3.411655e+06
4,4.950377e+15,2024-07-14 00:00:00.000 UTC,8.265621e-03,0.01%,0.540105,0x42161084d0672e1d3f26a9b53e653be2084ff19c,136,USDT-WETH,2.385039e+04,2.264554e-01,5.401049e+03
...,...,...,...,...,...,...,...,...,...,...,...
3463,4.005206e+14,2024-07-21 01:00:00.000 UTC,5.688889e-05,0.01%,0.002823,0x42161084d0672e1d3f26a9b53e653be2084ff19c,10,USDT-WETH,1.811063e+04,1.558600e-03,2.822722e+01
3464,2.830767e+17,2024-07-21 01:00:00.000 UTC,1.923425e-05,0.05%,2.789811,0x2f5e87c9312fa29aed5c179e456625d79015299c,11,WBTC-WETH,5.294103e+07,1.053932e-04,5.579623e+03
3465,1.792792e+14,2024-07-21 01:00:00.000 UTC,1.801927e-05,0.01%,0.001416,0x03a3be7ab4aa263d42d63b6cc594f4fb3d3f3951,2,WBTC-WETH,2.867966e+04,4.936787e-04,1.415854e+01
3466,2.355089e+21,2024-07-21 01:00:00.000 UTC,3.497822e-06,0.01%,0.012592,0x7cf803e8d82a50504180f417b8bc7a493c0a0503,4,DAI-USDC,1.313937e+06,9.583075e-05,1.259156e+02


In [63]:
filtered_cols = ['day','arbitrum_avg_liquidity','arbitrum_fees_usd','arbitrum_tvl_usd','arbitrum_volume_usd','arbitrum_num_trades','arbitrum_volume_to_tvl','arbitrum_token_pair']

In [100]:
arb_pool_df.head()

,arbitrum_avg_liquidity,day,arbitrum_fee_apr,arbitrum_fee_tier,arbitrum_fees_usd,arbitrum_lp_addr,arbitrum_num_trades,arbitrum_token_pair,arbitrum_tvl_usd,arbitrum_volume_to_tvl,arbitrum_volume_usd
0,1.854139e+23,2024-07-14 00:00:00.000 UTC,0.037815,0.30%,386.685410,0x4cef551255ec96d89fec975446301b5c4e164c59,119,WETH-ZRO,3.732390e+06,0.034534,1.288951e+05
1,2.075720e+16,2024-07-14 00:00:00.000 UTC,0.015918,0.01%,3.056607,0x6f38e884725a116c9c7fbf208e79fe8828a2595f,287,USDC-WETH,7.008980e+04,0.436099,3.056607e+04
2,1.965666e+12,2024-07-14 00:00:00.000 UTC,0.014695,0.05%,412.274720,0x0e4831319a50228b9e450861297ab92dee15b44f,466,USDC-WBTC,1.024001e+07,0.080522,8.245494e+05
3,4.911041e+18,2024-07-14 00:00:00.000 UTC,0.012910,0.05%,1705.827484,0xc6962004f452be9203591991d15f6b388e09e8d0,845,USDC-WETH,4.822924e+07,0.070738,3.411655e+06
4,4.950377e+15,2024-07-14 00:00:00.000 UTC,0.008266,0.01%,0.540105,0x42161084d0672e1d3f26a9b53e653be2084ff19c,136,USDT-WETH,2.385039e+04,0.226455,5.401049e+03


In [123]:
weth_zro_30 = arb_pool_df[(arb_pool_df['arbitrum_token_pair']=='WETH-ZRO') & (arb_pool_df['arbitrum_fee_tier']=='0.30%')]
# weth_zro = weth_zro[filtered_cols]
weth_zro_30.columns = [f'{col}_.30%' if col != 'day' else col for col in weth_zro_30.columns]
weth_zro_30.columns

Index(['arbitrum_avg_liquidity_.30%', 'day', 'arbitrum_fee_apr_.30%',
       'arbitrum_fee_tier_.30%', 'arbitrum_fees_usd_.30%',
       'arbitrum_lp_addr_.30%', 'arbitrum_num_trades_.30%',
       'arbitrum_token_pair_.30%', 'arbitrum_tvl_usd_.30%',
       'arbitrum_volume_to_tvl_.30%', 'arbitrum_volume_usd_.30%'],
      dtype='object')

In [122]:
weth_zro_fig = make_subplots(specs=[[{"secondary_y": True}]])

weth_zro_fig.add_trace(
    go.Bar(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_avg_liquidity'],
        name='Liquidity'
    ),
    secondary_y=False
)

weth_zro_fig.add_trace(
    go.Scatter(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_num_trades'],
        name='# Of Trades',
        mode='lines'
    ),
    secondary_y=True
)

weth_zro_fig.update_layout(title='Liquidity to # of Trades Comparison')

weth_zro_fig.update_xaxes(title_text="Date")

weth_zro_fig.show()


In [105]:
corr_matrix = weth_zro_30[['arbitrum_avg_liquidity', 'arbitrum_fees_usd', 'arbitrum_tvl_usd', 'arbitrum_volume_usd']].corr()

# Create a heatmap
fig = go.Figure(data=go.Heatmap(
                   z=corr_matrix.values,
                   x=corr_matrix.columns,
                   y=corr_matrix.columns,
                   colorscale='Viridis',
                   zmin=-1, zmax=1))

fig.update_layout(
    title='Correlation Matrix',
    xaxis_nticks=36
)

fig.show()

In [76]:
arbitrum_gas_df.rename(columns={'dt':'day'}, inplace=True)

In [106]:
weth_zro_30 = weth_zro_30.merge(arbitrum_gas_df, how='left', on=['day']) 


In [107]:
weth_zro_30.head()

,arbitrum_avg_liquidity,day,arbitrum_fee_apr,arbitrum_fee_tier,arbitrum_fees_usd,arbitrum_lp_addr,arbitrum_num_trades,arbitrum_token_pair,arbitrum_tvl_usd,arbitrum_volume_to_tvl,arbitrum_volume_usd,arbitrum_gas_usd_per_tx,arbitrum_median_gas_usd
0,1.854139e+23,2024-07-14 00:00:00.000 UTC,0.037815,0.30%,386.685410,0x4cef551255ec96d89fec975446301b5c4e164c59,119,WETH-ZRO,3.732390e+06,0.034534,128895.136815,0.010340,0.006240
1,1.760607e+23,2024-07-14 01:00:00.000 UTC,0.023862,0.30%,243.510673,0x4cef551255ec96d89fec975446301b5c4e164c59,77,WETH-ZRO,3.724752e+06,0.021792,81170.224269,0.007959,0.005059
2,1.613444e+23,2024-07-14 02:00:00.000 UTC,0.041648,0.30%,421.222300,0x4cef551255ec96d89fec975446301b5c4e164c59,126,WETH-ZRO,3.691604e+06,0.038034,140407.433278,0.005898,0.004483
3,1.508496e+23,2024-07-14 03:00:00.000 UTC,0.033474,0.30%,337.849127,0x4cef551255ec96d89fec975446301b5c4e164c59,118,WETH-ZRO,3.683938e+06,0.030570,112616.375723,0.005264,0.004377
4,1.610693e+23,2024-07-14 04:00:00.000 UTC,0.069065,0.30%,700.431649,0x4cef551255ec96d89fec975446301b5c4e164c59,237,WETH-ZRO,3.701668e+06,0.063074,233477.216492,0.006068,0.004599


In [108]:
weth_zro_30['arbitrum_net_liquidity'] = weth_zro_30['arbitrum_avg_liquidity'] - weth_zro_30['arbitrum_avg_liquidity'].shift(1)
weth_zro_30['arbitrum_net_liquidity'].fillna(0, inplace=True) 
print(weth_zro_30[['day', 'arbitrum_avg_liquidity', 'arbitrum_net_liquidity']].head())

                           day  arbitrum_avg_liquidity  arbitrum_net_liquidity
0  2024-07-14 00:00:00.000 UTC            1.854139e+23            0.000000e+00
1  2024-07-14 01:00:00.000 UTC            1.760607e+23           -9.353201e+21
2  2024-07-14 02:00:00.000 UTC            1.613444e+23           -1.471624e+22
3  2024-07-14 03:00:00.000 UTC            1.508496e+23           -1.049480e+22
4  2024-07-14 04:00:00.000 UTC            1.610693e+23            1.021968e+22


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_47392\1579211113.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [110]:
weth_zro_gas_to_trades_fig = make_subplots(specs=[[{"secondary_y": True}]])

weth_zro_gas_to_trades_fig.add_trace(
    go.Bar(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_num_trades'],
        name='# of Trades'
    ),
    secondary_y=False
)

weth_zro_gas_to_trades_fig.add_trace(
    go.Scatter(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_median_gas_usd'],
        name='Median Gas',
        mode='lines'
    ),
    secondary_y=True
)

weth_zro_gas_to_trades_fig.add_trace(
    go.Scatter(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_gas_usd_per_tx'],
        name='Avg Gas',
        mode='lines'
    ),
    secondary_y=True
)

weth_zro_gas_to_trades_fig.update_layout(title='Trades to Gas Comparison')

weth_zro_gas_to_trades_fig.update_xaxes(title_text="Date")

weth_zro_gas_to_trades_fig.show()


In [111]:
weth_zro_30.columns

Index(['arbitrum_avg_liquidity', 'day', 'arbitrum_fee_apr',
       'arbitrum_fee_tier', 'arbitrum_fees_usd', 'arbitrum_lp_addr',
       'arbitrum_num_trades', 'arbitrum_token_pair', 'arbitrum_tvl_usd',
       'arbitrum_volume_to_tvl', 'arbitrum_volume_usd',
       'arbitrum_gas_usd_per_tx', 'arbitrum_median_gas_usd',
       'arbitrum_net_liquidity'],
      dtype='object')

In [116]:
weth_zro_gas_to_liq_fig = make_subplots(specs=[[{"secondary_y": True}]])


weth_zro_gas_to_liq_fig.add_trace(
    go.Bar(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_avg_liquidity'],
        name='Liquidity'
    ),
    secondary_y=False
)

weth_zro_gas_to_liq_fig.add_trace(
    go.Scatter(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_median_gas_usd'],
        name='Median Gas',
        mode='lines'
    ),
    secondary_y=True
)

weth_zro_gas_to_liq_fig.add_trace(
    go.Scatter(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_gas_usd_per_tx'],
        name='Avg Gas',
        mode='lines'
    ),
    secondary_y=True
)

weth_zro_gas_to_liq_fig.update_layout(title='Liquidity to Gas Comparison')

weth_zro_gas_to_liq_fig.update_xaxes(title_text="Date")

weth_zro_gas_to_liq_fig.show()


In [120]:
weth_zro_gas_to_net_liq_fig = make_subplots(specs=[[{"secondary_y": True}]])


weth_zro_gas_to_net_liq_fig.add_trace(
    go.Bar(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_net_liquidity'],
        name='Net Liquidity Added'
    ),
    secondary_y=False
)

weth_zro_gas_to_net_liq_fig.add_trace(
    go.Scatter(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_median_gas_usd'],
        name='Median Gas',
        mode='lines'
    ),
    secondary_y=True
)

weth_zro_gas_to_net_liq_fig.add_trace(
    go.Scatter(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_gas_usd_per_tx'],
        name='Avg Gas',
        mode='lines'
    ),
    secondary_y=True
)

weth_zro_gas_to_net_liq_fig.update_layout(title='Net Liquidity to Gas Comparison')

weth_zro_gas_to_net_liq_fig.update_xaxes(title_text="Date")

weth_zro_gas_to_net_liq_fig.show()


In [121]:
weth_zro_gas_to_vtt_fig = make_subplots(specs=[[{"secondary_y": True}]])


weth_zro_gas_to_vtt_fig.add_trace(
    go.Bar(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_volume_to_tvl'],
        name='Volume to TVL Ratio'
    ),
    secondary_y=False
)

weth_zro_gas_to_vtt_fig.add_trace(
    go.Scatter(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_median_gas_usd'],
        name='Median Gas',
        mode='lines'
    ),
    secondary_y=True
)

weth_zro_gas_to_vtt_fig.add_trace(
    go.Scatter(
        x=weth_zro_30['day'],
        y=weth_zro_30['arbitrum_gas_usd_per_tx'],
        name='Avg Gas',
        mode='lines'
    ),
    secondary_y=True
)

weth_zro_gas_to_vtt_fig.update_layout(title='Volume/TVL Ratio to Gas Comparison')

weth_zro_gas_to_vtt_fig.update_xaxes(title_text="Date")

weth_zro_gas_to_vtt_fig.show()


## Optimism

## Base

## Trader Behavior

# Correlation Analysis